In [1]:
import numpy as np
import MyML.helper.partition as myPart
import MyML.cluster.K_Means3 as myKM
import MyML.utils.sparse as mySparse
import MyML.cluster.eac as eac
from sklearn.datasets import make_blobs

/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/cuda/decorators.py:106: UserWarning: autojit is deprecated and will be removed in a future release. Use jit instead.
  warn('autojit is deprecated and will be removed in a future release. Use jit instead.')


In [2]:
reload(mySparse)

<module 'MyML.utils.sparse' from '/home/chiroptera/workspace/QCThesis/MyML/utils/sparse.pyc'>

In [4]:
data,gt = make_blobs(n_samples=5000, n_features=2, centers=6)
data = data.astype(np.float32)
n_samples = data.shape[0]

In [5]:
generator = myKM.K_Means(cuda_mem="manual")
generator._MAX_THREADS_BLOCK = 256

In [6]:
sqrt_rule = lambda n: map(int, map(np.ceil, [np.sqrt(n) / 2, np.sqrt(n)]))
n_clusters = sqrt_rule(data.shape[0])

In [7]:
%time ensemble = myPart.generateEnsemble(data, generator, n_clusters, npartitions=100)

CPU times: user 1.77 s, sys: 112 ms, total: 1.89 s
Wall time: 1.88 s


In [8]:
bgs = mySparse._compute_max_assocs_from_ensemble(ensemble)
max_assocs = bgs * 3
print max_assocs

1242


In [ ]:
mat0 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
%time mat0._update_ensemble(ensemble)
%time mat0._condense()

In [9]:
mat1 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat1.sort_mode = "online"
mat1.update_cluster_function = mySparse.update_cluster_sorted
%time mat1._update_ensemble(ensemble)
%time mat1._condense()

CPU times: user 7.71 s, sys: 23.4 ms, total: 7.73 s
Wall time: 7.74 s
CPU times: user 193 ms, sys: 3.93 ms, total: 196 ms
Wall time: 197 ms


In [140]:
mat2 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat2.sort_mode = "online"
mat2.update_cluster_function = mySparse.update_cluster_sorted_simple
%debug mat2._update_ensemble(ensemble)
%time mat2._condense()

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

ipdb> b MyML/utils/sparse.py:57
Breakpoint 1 at /home/chiroptera/workspace/QCThesis/MyML/utils/sparse.py:57
ipdb> c
> /home/chiroptera/workspace/QCThesis/MyML/utils/sparse.py(57)update_partition()
     56 
1--> 57         if self.sort_mode == "numpy":
     58                 self._sort_indices()

ipdb> new_nnz_list
[0, 48378, 26920, 0, 27638, 5770, 10228, 5310, 2990, 20884, 8136, 544, 10618, 3840, 18692, 934, 0, 16894, 3130, 11812, 1708, 180, 480, 10304, 9850, 2548, 102, 2300, 1710, 3040, 836, 8268, 600, 0, 4588, 10896, 11966, 6864, 5206, 17700, 68, 3266, 2112, 440, 3020, 3778, 2346, 0, 292]
ipdb> c
> /home/chiroptera/workspace/QCThesis/MyML/utils/sparse.py(57)update_partition()
     56 
1--> 57         if self.sort_mode == "numpy":
     58                 self._sort_indices()

ipdb> new_nnz_list
[0, 178, 0, 7944, 104, 116, 576, 2914, 898, 0, 420, 574, 286, 304, 506, 3100, 144, 3724, 5498, 0, 1098, 201

In [ ]:
b MyML/utils/sparse.py:57

In [146]:
print map(max,nnz2)
print map(max,nnz3)

[48378, 22066, 17360, 62304]
[48378, 23751, 27134, 84705]


In [143]:
nnz2 = [
    [0, 48378, 26920, 0, 27638, 5770, 10228, 5310, 2990, 20884, 8136, 544, 10618, 3840, 18692, 934, 0, 16894, 3130, 11812, 1708, 180, 480, 10304, 9850, 2548, 102, 2300, 1710, 3040, 836, 8268, 600, 0, 4588, 10896, 11966, 6864, 5206, 17700, 68, 3266, 2112, 440, 3020, 3778, 2346, 0, 292],
    [0, 178, 0, 7944, 104, 116, 576, 2914, 898, 0, 420, 574, 286, 304, 506, 3100, 144, 3724, 5498, 0, 1098, 20164, 22066, 0, 2006, 528, 268, 0, 54, 1526, 440, 198, 0, 224, 658, 2354, 1998, 2858, 3758, 0, 1076, 28, 1884, 1358, 644, 0, 4778, 5710, 2924, 104, 136, 2808, 1504, 72, 682, 918, 1110, 0, 0, 1036, 0, 0, 240, 0, 222, 172],
    [0, 0, 520, 2456, 0, 282, 138, 0, 906, 4376, 1116, 710, 284, 172, 0, 4342, 0, 128, 9484, 0, 140, 426, 480, 4498, 1426, 0, 0, 0, 4116, 1272, 0, 1520, 0, 1340, 3210, 0, 78, 818, 0, 788, 436, 0, 3682, 1596, 3000, 326, 2156, 6838, 6002, 10032, 1320, 5018, 1246, 996, 1268, 17360, 3500],
    [0, 226, 52882, 0, 120, 0, 1062, 220, 58, 244, 1516, 32, 6294, 564, 370, 370, 180, 228, 144, 490, 38, 62304, 60, 372, 448, 64, 1050, 4, 288, 160, 436, 0, 472, 2266, 6190, 588, 804, 7236, 868, 1660, 566, 26, 344, 256, 572, 0, 0, 12, 974, 0, 2008, 1606, 10482]
]

In [142]:
nnz3 = [
    [0, 48378, 26920, 0, 27638, 5770, 10228, 5310, 2990, 20884, 8136, 544, 10618, 3840, 18692, 934, 0, 16894, 3130, 11812, 1708, 180, 480, 10304, 9850, 2548, 102, 2300, 1710, 3040, 836, 8268, 600, 0, 4588, 10896, 11966, 6864, 5206, 17700, 68, 3266, 2112, 440, 3020, 3778, 2346, 0, 292],
    [1445, 2209, 0, 11207, 257, 289, 648, 7108, 1086, 0, 612, 706, 775, 913, 506, 3452, 592, 3957, 7707, 0, 1687, 23751, 23287, 776, 2073, 657, 380, 492, 198, 2469, 559, 230, 1748, 383, 1297, 2382, 2056, 4117, 4445, 250, 1245, 28, 2228, 2120, 661, 470, 4804, 6725, 3949, 117, 1834, 3851, 1684, 276, 1303, 984, 1331, 1436, 1442, 1659, 52, 26, 249, 2797, 295, 188],
    [1940, 561, 1283, 3789, 405, 2298, 7890, 448, 4843, 8488, 1586, 1996, 3994, 1698, 6991, 6612, 153, 1831, 18007, 85, 306, 4734, 1087, 6778, 2812, 1561, 424, 1834, 5598, 3235, 2898, 4233, 3259, 3162, 3632, 3253, 618, 1792, 4161, 1209, 2221, 545, 9031, 6614, 7417, 1007, 4384, 12048, 7772, 13411, 3447, 6668, 2470, 2478, 2595, 27134, 7254],
    [91, 6366, 68673, 697, 7975, 39, 5810, 1674, 1099, 3139, 3149, 471, 14740, 2068, 1633, 6038, 5457, 2588, 2707, 1281, 1147, 84705, 327, 1944, 5203, 352, 4328, 2308, 3414, 1620, 5633, 7827, 2631, 4247, 8699, 6755, 5911, 11066, 4590, 3493, 2351, 333, 3916, 2126, 2742, 3928, 3694, 16, 2524, 970, 6562, 3585, 17006]
]

True

In [198]:
reload(mySparse)
mat3 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat3.sort_mode = "online"
mat3.update_cluster_function = mySparse.update_cluster_sorted_surgical
%time mat3._update_ensemble(ensemble)

CPU times: user 7.79 s, sys: 11.6 ms, total: 7.8 s
Wall time: 7.8 s


In [ ]:
b MyML/utils/sparse.py:360
start_idx, end_idx, n_shifts
new_assocs_ptr
new_assocs_ids[:new_assocs_ptr+1]
new_assocs_idx[:new_assocs_ptr+1]
data[[idx,idx+n_shifts]], indices[[idx,idx+n_shifts]]

In [79]:
print "indices", (mat1.indices==mat2.indices).all()
print "data", (mat1.data==mat2.data).all()
print "degree", (mat1.degree==mat2.degree).all()

indices True
data False
degree True


In [36]:
mat3.degree

array([2502, 2502, 2502, ..., 2502, 2502,    0], dtype=int32)

In [224]:
reload(mySparse)

<module 'MyML.utils.sparse' from '/home/chiroptera/workspace/QCThesis/MyML/utils/sparse.py'>

In [225]:
%%timeit
mat1 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat1.sort_mode = "online"
mat1.update_cluster_function = mySparse.update_cluster_sorted
mat1._update_ensemble(ensemble)

1 loops, best of 3: 9.85 s per loop


In [226]:
%%timeit 
mat2 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat2.sort_mode = "online"
mat2.update_cluster_function = mySparse.update_cluster_sorted_simple
mat2._update_ensemble(ensemble)

1 loops, best of 3: 9.82 s per loop


In [227]:
%%timeit 
mat3 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat3.sort_mode = "online"
mat3.update_cluster_function = mySparse.update_cluster_sorted_surgical
mat3._update_ensemble(ensemble)

1 loops, best of 3: 9.14 s per loop


In [185]:
mat0 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
%time mat0._update_ensemble(ensemble)

mat1 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat1.sort_mode = "online"
mat1.update_cluster_function = mySparse.update_cluster_sorted
%time mat1._update_ensemble(ensemble)

mat2 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat2.sort_mode = "online"
mat2.update_cluster_function = mySparse.update_cluster_sorted_simple
%time mat2._update_ensemble(ensemble)

mat3 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat3.sort_mode = "online"
mat3.update_cluster_function = mySparse.update_cluster_sorted_surgical
%time mat3._update_ensemble(ensemble)

CPU times: user 15.1 s, sys: 27.2 ms, total: 15.1 s
Wall time: 15.4 s
CPU times: user 7.51 s, sys: 28.1 ms, total: 7.54 s
Wall time: 7.57 s
CPU times: user 7.31 s, sys: 15.6 ms, total: 7.33 s
Wall time: 7.36 s


TypingError: Caused By:
Traceback (most recent call last):
  File "/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/compiler.py", line 201, in run
    res = stage()
  File "/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/compiler.py", line 415, in stage_nopython_frontend
    self.locals)
  File "/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/compiler.py", line 709, in type_inference_stage
    infer.build_constrain()
  File "/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 395, in build_constrain
    self.constrain_statement(inst)
  File "/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 519, in constrain_statement
    self.typeof_assign(inst)
  File "/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 551, in typeof_assign
    self.typeof_global(inst, inst.target, value)
  File "/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/typeinfer.py", line 616, 

In [206]:
setup1 = """import MyML.utils.sparse as mySparse
mat1 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat1.sort_mode = "online"
mat1.update_cluster_function = mySparse.update_cluster_sorted"""

In [165]:
print mat2.new_nnz_list[1]
print mat3.new_nnz_list[1]

[3826, 4166, 8104, 262, 7130, 80, 17168, 1862, 528, 720, 704, 4016, 2088, 1780, 2224, 710, 3644, 15364, 10480, 15918, 0, 25786, 12662, 0, 1628, 23650, 4200, 1116, 5918, 0, 9282, 1946, 14522, 3060, 4856, 9420, 1560, 152, 1818]
[3974, 7101, 8368, 1555, 10766, 82, 17452, 10889, 605, 1306, 4752, 4455, 2293, 1864, 2475, 873, 9293, 21415, 15219, 24665, 79, 27261, 13131, 0, 1782, 27013, 5335, 1209, 10117, 120, 11945, 2112, 20473, 3263, 5062, 11091, 2067, 219, 6458]


In [188]:
%time mat0._condense()
%time mat1._condense()
%time mat2._condense()
%time mat3._condense()

CPU times: user 309 ms, sys: 4.01 ms, total: 313 ms
Wall time: 308 ms
CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 11.6 ms
CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 11.4 ms
CPU times: user 17.1 ms, sys: 14 µs, total: 17.1 ms
Wall time: 17.3 ms


In [189]:
eacEst = eac.EAC(n_samples=n_samples)
%time eacEst.fit(ensemble)
eacEst._getAssocsDegree()

CPU times: user 620 ms, sys: 4.01 ms, total: 624 ms
Wall time: 621 ms


In [192]:
reload(mySparse)
mat3 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat3.sort_mode = "online"
mat3.update_cluster_function = mySparse.update_cluster_sorted_surgical
%time mat3._update_ensemble(ensemble)
mat3._condense()

CPU times: user 8.22 s, sys: 35.8 ms, total: 8.25 s
Wall time: 8.23 s


In [199]:
print "check matrices"
print "mat0: ", (eacEst._coassoc == mat0.todense()).all()
print "mat1: ", (eacEst._coassoc == mat1.todense()).all()
print "mat2: ", (eacEst._coassoc == mat2.todense()).all()
print "mat3: ", (eacEst._coassoc == mat3.todense()).all()

check matrices
mat0:  True
mat1:  True
mat2:  True
mat3:  False


In [200]:
print "eac:  ", eacEst.nnz
print "mat0: ",mat0.nnz
print "mat1: ",mat1.nnz
print "mat2: ",mat2.nnz
print "mat3: ",mat3.nnz

eac:   2976590
mat0:  2976590
mat1:  2976590
mat2:  2976590
mat3:  4891864


# Test linkages

In [1]:
import MyML.metrics.accuracy as myAcc
import numpy as np
import MyML.helper.partition as myPart
import MyML.cluster.K_Means3 as myKM
import MyML.utils.sparse as mySparse
import MyML.cluster.eac as eac
from sklearn.datasets import make_blobs

/home/chiroptera/anaconda/lib/python2.7/site-packages/numba/cuda/decorators.py:106: UserWarning: autojit is deprecated and will be removed in a future release. Use jit instead.
  warn('autojit is deprecated and will be removed in a future release. Use jit instead.')


In [2]:
data,gt = make_blobs(n_samples=5000, n_features=2, centers=6)
data = data.astype(np.float32)
n_samples = data.shape[0]

generator = myKM.K_Means(cuda_mem="manual")
generator._MAX_THREADS_BLOCK = 256

sqrt_rule = lambda n: map(int, map(np.ceil, [np.sqrt(n) / 2, np.sqrt(n)]))
n_clusters = sqrt_rule(data.shape[0])

%time ensemble = myPart.generateEnsemble(data, generator, n_clusters, npartitions=100)

bgs = mySparse._compute_max_assocs_from_ensemble(ensemble)
max_assocs = bgs * 3
print "max assocs", max_assocs

mat1 = mySparse.EAC_CSR(n_samples=n_samples, max_assocs=max_assocs)
mat1.sort_mode = "online"
mat1.update_cluster_function = mySparse.update_cluster_sorted
%time mat1._update_ensemble(ensemble)
%time mat1._condense()

eacEst = eac.EAC(n_samples=n_samples)
%time eacEst.fit(ensemble)

print "equal mats", (eacEst._coassoc == mat1.todense()).all()
print mat1.nnz

CPU times: user 1.86 s, sys: 148 ms, total: 2.01 s
Wall time: 2.01 s
max assocs 2499
CPU times: user 7.57 s, sys: 12.2 ms, total: 7.58 s
Wall time: 7.56 s
CPU times: user 223 ms, sys: 0 ns, total: 223 ms
Wall time: 223 ms
CPU times: user 810 ms, sys: 20 ms, total: 829 ms
Wall time: 829 ms
equal mats True
3316272


In [3]:
import MyML.cluster.linkage as myLinkage
reload(myLinkage)

<module 'MyML.cluster.linkage' from '/home/chiroptera/workspace/QCThesis/MyML/cluster/linkage.pyc'>

In [4]:
eacEst = eac.EAC(n_samples=n_samples)
%time eacEst.fit(ensemble)
%time labels = eacEst._lifetime_clustering()
accScorer = myAcc.HungarianIndex(nsamples=data.shape[0])
accScorer.score(gt, labels)
print accScorer.accuracy

CPU times: user 629 ms, sys: 8.21 ms, total: 637 ms
Wall time: 627 ms
CPU times: user 1.14 s, sys: 249 ms, total: 1.39 s
Wall time: 1.38 s
0.1668


In [9]:
dest = mat1.indices
weight = mat1.data.max() + 1 - mat1.data
fe = mat1.indptr[:-1]
od = mat1.indptr[1:]

In [11]:
dest.size / fe.size * 1.0

663.0

In [12]:
reload(myLinkage)
%time myLinkage.sl_mst_lifetime_seq(dest, weight, fe, od, disconnect_weight = None)

CPU times: user 51.5 s, sys: 0 ns, total: 51.5 s
Wall time: 51.7 s


array([   0,    1,    2, ..., 4974, 4975, 4976], dtype=int32)

In [7]:
import MyML.graph.mst as myMST

In [8]:
myMST.boruvka_minho_seq(dest, weight, fe, od)

(array([  92, 1148, 2274, ...,    0,    0,    0], dtype=int32), 4999)

In [ ]:
myLinkage.sl_mst_lifetime_gpu(dest, weight, fe, od, disconnect_weight = None)

In [ ]:
b MyML/cluster/linkage.py:24